In [ ]:
import numpy as np
import alluvion as al
from matplotlib import pyplot as plt
from util import populate_plt_settings, get_column_width, get_fig_size, get_latex_float

In [ ]:
dp = al.Depot(np.float32)
runner = dp.Runner()

In [ ]:
def calculate_masked_mse(v1, v2, mask):
    v_diff = v1 - v2
    masked_se = np.sum(v_diff * v_diff, axis=1) * mask
    num_masked = np.sum(mask)
    return np.sum(masked_se)/num_masked if num_masked>0 else 0

In [ ]:
truth_dirs = ['/media/kennychufk/vol1bk0/20210415_142855/',
             '/media/kennychufk/vol1bk0/20210415_144021/',
             '/media/kennychufk/vol1bk0/20210415_145307/',
             '/media/kennychufk/vol1bk0/20210415_150358/',
             '/media/kennychufk/vol1bk0/20210415_151554/',
             '/media/kennychufk/vol1bk0/20210415_152942/'
             ]
v2_list = []
for truth_dir in truth_dirs:
    truth_v_piv = np.load(f'{truth_dir}/mat_results/vel_filtered.npy')
    num_samples = truth_v_piv.shape[1]*truth_v_piv.shape[2]
    truth_v_piv = truth_v_piv.reshape(-1, num_samples ,2)
    mask_np = np.load(f'{truth_dir}/mat_results/mask-raisevlim.npy').reshape(-1, num_samples).astype(np.float32)

    truth_v_np = np.zeros((*truth_v_piv.shape[:-1], 3))
    truth_v_np[..., 2] = truth_v_piv[..., 0]
    truth_v_np[..., 1] = truth_v_piv[..., 1]
    
    ground_truth = dp.create_coated((num_samples), 3)
    zero3 = dp.create_coated_like(ground_truth)
    zero3.set_zero()
    mask = dp.create_coated((num_samples), 1)
    v2 = np.zeros(len(truth_v_piv))
    for frame_id in range(len(truth_v_piv)):
        mask.set(mask_np[frame_id])
        ground_truth.set(truth_v_np[frame_id])
        v2[frame_id] = runner.calculate_se_yz_masked(zero3, ground_truth, mask,num_samples)/np.sum(mask_np[frame_id])
    v2_list.append(v2)
    
    dp.remove(ground_truth)
    dp.remove(zero3)

In [ ]:
truth_dirs = ['/media/kennychufk/vol1bk0/20210415_142855/',
             '/media/kennychufk/vol1bk0/20210415_144021/',
             '/media/kennychufk/vol1bk0/20210415_145307/',
             '/media/kennychufk/vol1bk0/20210415_150358/',
             '/media/kennychufk/vol1bk0/20210415_151554/',
             '/media/kennychufk/vol1bk0/20210415_152942/'
             ]
v2_list = []
for truth_dir in truth_dirs:
    truth_v_piv = np.load(f'{truth_dir}/mat_results/vel_filtered.npy')
    num_samples = truth_v_piv.shape[1]*truth_v_piv.shape[2]
    truth_v_piv = truth_v_piv.reshape(-1, num_samples ,2)
    np.nan_to_num(truth_v_piv, copy=False)
    mask_np = np.load(f'{truth_dir}/mat_results/mask-raisevlim.npy').reshape(-1, num_samples)
    zero_vector =np.zeros_like(truth_v_piv[0])
    v2 = np.zeros(len(truth_v_piv))
    
    for frame_id in range(len(truth_v_piv)):
        v2[frame_id] = np.sqrt(calculate_masked_mse(truth_v_piv[frame_id], zero_vector, mask_np[frame_id]))
    v2_list.append(v2)
    

In [ ]:
plt.plot(v2_list[1])

In [ ]:
populate_plt_settings(plt)

In [ ]:
#matplotlib.rcParams['axes.unicode_minus'] = False
num_rows = 1
num_cols = 1
fig, ax = plt.subplots(num_rows, num_cols, figsize = get_fig_size(get_column_width()))
cmap = plt.get_cmap("tab10")

for i, v2 in enumerate(v2_list):
#     ax.plot((np.arange(1000)+2000)/500,v2[2000:3000], label=f'{i+4} buoys')
    ax.plot((np.arange(3000)+400)/500,v2[400:3400], label=f'{i+4} buoys', linewidth=0.8)
#     plt.plot(np.convolve(v2, np.ones(50)/30)[4000:6000], label=f'{i+4}')
#     print(np.sqrt(np.mean(v2)))
    ax.set_xlabel('Time (s)')
    ax.set_ylabel('Root-mean-square speed (m/s)')
ax.legend()
fig.tight_layout(pad=0.05) # pad is 1.08 by default https://stackoverflow.com/a/59252633
fig.savefig('buoy-effect-on-flow.pgf', bbox_inches='tight') # bbox_inches='tight' necessary for keeping the time legend inside the canvas